# Cleaning Sequences
There are multiple ways of sequencing the steps in your data cleaning process. We've used one so far, once in Lesson 1 and once in the **Data Cleaning Process** example video in this lesson. The **Define**, **Code**, and **Test** Markdown headers were used once in this sequence, with multiple definitions, cleaning operations, and tests under each header, respectively.

It looked like this:

______

## Gather

In [69]:
import pandas as pd
import requests
import tweepy
import timeit
import json

#### Keys

In [8]:
consumer_key = 'wp5lVM9qitFOplu1dWeQ34x26' 
consumer_secret = 'iXPxPQ0HEO14U8wMwwiLugMwZbpeadPWRbGSjNFI3jHriwQmKA'

access_token = '739672993-sESdNtfAq523OsJduxGBUmUnZLSfz4274y7E3eCn' 
access_token_secret = 'vqYxHjS2sEPRfRHjdEv145h5WbtgtVQiTh46krp3ZB4FC' 

In [9]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
req = requests.get(url, auth=('user','pass'));
open('image_predictions.tsv', 'wb').write(req.content);

In [10]:
img_pred = pd.read_csv('image_predictions.tsv', sep='\t');

In [11]:
img_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [41]:
tweet_archive = pd.read_csv('twitter-archive-enhanced.csv');

In [47]:
tweet_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [48]:
#Query Twitter API for Tweet JSON data
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#Call API
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


#api.get_status(tweet_id, tweet_mode='extended')

In [51]:
num_tweets = 200
start = timeit.timeit()
tweets = []
deleted_tweets = []
#for tweet in tweepy.Cursor(api.user_timeline, screen_name='dog_rates').items(num_tweets):
#        tweets.append(tweet)
for tweet_id in tweet_archive.tweet_id:
    try:
        tweet = api.get_status(id=tweet_id)
        tweets.append(tweet)
    except tweepy.TweepError:
        deleted_tweets.append(tweet_id)
end = timeit.timeit()
print("Gathering Tweets took a total of ", end - start)

Rate limit reached. Sleeping for: 656
Rate limit reached. Sleeping for: 806


Gathering Tweets took a total of  -0.0006825880000178586


In [93]:
with open("tweet_json.txt","w") as file:
    for tweet in tweets:
        json.dump(tweet._json, file, ensure_ascii=False)
        file.write('\n')
#tmp = [[tweet.id_str, tweet.retweet_count, tweet.favorite_count, tweet.text.encode('utf-8')] for tweet in tweets]
#df = pd.read_csv(tweets_for_csv)


In [156]:
tweet_data = pd.DataFrame({}, columns =['tweet_id', 'description','retweet_count', 'favorite_count'])
json_data = []
with open("tweet_json.txt", "r") as file:
    for line in file:
        json_data = json.loads(line)
        df2 = pd.DataFrame({"tweet_id":[json_data['id']],
                            "description":[json_data["text"]],
                           "retweet_count": [json_data['retweet_count']],
                           'favorite_count': [json_data['favorite_count']]})
        tweet_data = tweet_data.append(df2, ignore_index=True)


In [157]:
tweet_data.head()

Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj


,tweet_id,description,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,7923,37097
1,892177421306343426,This is Tilly. She's just checking pup on you....,5886,31916
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,3888,24031
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8080,40416
4,891327558926688256,This is Franklin. He would like you to stop ca...,8759,38641


## Assess

#### Quality
- 

## Clean

#### Define
- Remove 'bb' before every animal name using string slicing
- Replace ! with . in body weight and brain weight columns

#### Code

#### Test

________

But you can also use multiple **Define**, **Code**, and **Test** headers, one for each data quality and tidiness issue (or group of data quality and tidiness issues). Effectively, you are defining then coding then testing immediately. This sequence is helpful when you have a lot of quality and tidiness issues to clean. Since that is the case in this lesson, this sequence will be used.

Pasting each assessment above the **Define** header as its own header can also be helpful.

Here's what this sequence looks like using the *animals.csv* dataset (and reusing the above *Gather* and *Assess* steps):

_______

## Clean

#### bb before every animal name
##### Define
Remove 'bb' before every animal name using string slicing.

##### Code

##### Test

#### ! instead of . for decimal in body weight and brain weight
##### Define
Replace ! with . in body weight and brain weight columns

##### Code

##### Test